# E-commerce Recommandation

someone searched laptop and brought in past. lets say that person is X
now, He might need charging stations, monitors, and warranties etc . He might also be shopping for his school kid and need other laptops and electronics. Knowledge of the customer’s desires and situations and up-selling,cross-selling additional items is easy money for any business.
If we can predict X’s shopping list, then we can sell him a lot more. More sales means more profit.
now up-selling means selling additional items that complement the items purchased. like if he brought laptop then up-selling would be laptop bag, mouse etc.
now cross-selling is the product which may liked by that person X as laptop likely to be brought by technical person so that person would like Mobile phone or someother technogical devices/gadgets.
also we need to counter The gender, along with occupation, age etc factors are really important we're dealing with recommending a product to customer.

In customer relation system we need to firstly identify that few things like
1. Customer Life Cycle.
2. Recommending the Best Channels for Contact.
3. Customer Lifetime Value.
4. Clustering and Predictive Analysis to group problems.
5. Recommending Items to Customers.

But these 5 processes/dependencies of whole process of predictive analytics for a customer whether he/she will stick to our business/services or not. But I’ll stick to that problem statement which is recommending item to customers.
Here is the data( ratings.csv ) which I have create looks like this with User IDs - ID of customers
item IDs - Items they brought
so we’ll try to recommend based on their past item brought.

## Codes along with details below

In [ ]:
#Importing Pandas for loading the data
import pandas as pd

In [4]:
#loading the data
userItemData = pd.read_csv('ratings.csv')

In [5]:
#checking the head of the dataframe
userItemData.head()

userId  ItemId
0    1001    5001
1    1001    5002
2    1001    5005
3    1002    5003
4    1002    5004

So, data file is very simple, which user bought which item. So, we have user IDs and item IDs. 
So users 1,001 seem to have bought 5,001, 5,002, and 5,005, and so on, pretty simple, straightforward data file. 
What we’re going to do here is, first, we’re going to load the file. Now we have to build an affinity score between items based on the users who bought them Now there are a number of out-of-the-box open source as well as commercial collaborative filtering libraries that can build this affinity score.


Now we are going to take every item and find the affinity of that item to other items, and the way I’m going to do it is by finding out how many customers have bought both these products. The more number of customers who buy both the products, the higher is going to be the affinity score. So that is how the affinity score I am going to be computing here, and after that you can see that I am printing out the affinity score between each items. Item one to item two has a high affinity score of .4,  whereas 5,001 to 5,003, there is no affinity at all.

In [6]:
#list of unique items
itemList=list(set(userItemData["ItemId"].tolist()))

In [7]:
#count of users
userCount=len(set(userItemData["ItemId"].tolist()))

In [8]:
#Creating an empty data frame to store item affinity scores for items.
itemAffinity= pd.DataFrame(columns=('item1', 'item2', 'score'))
rowCount=0

In [26]:
for ind1 in range(len(itemList)):
    
    #list of users who bought this item 1.
    item1Users = userItemData[userItemData.ItemId==itemList[ind1]]["userId"].tolist()

    #we can alos print this line too see item1 associated with which userId
    #print("Item 1 ", item1Users)
    
    #item 2 - items that are not item 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(itemList)): 
        
        if ( ind1 == ind2):
            continue
       
        #list of users who bought item 2
        item2Users=userItemData[userItemData.ItemId==itemList[ind2]]["userId"].tolist()
        
        #we can alos print this line too see item2 associated with which userId
        #print("Item 2",item2Users)
        
        
        #Finding score.common list of users and divide it by the total users.
        commonUsers= len(set(item1Users).intersection(set(item2Users)))
        score=commonUsers / userCount

        #Adding a score for item 1, item 2
        itemAffinity.loc[rowCount] = [itemList[ind1],itemList[ind2],score]
        rowCount +=1
        #Add a score for item2, item 1. The same score would apply irrespective of the sequence.
        itemAffinity.loc[rowCount] = [itemList[ind2],itemList[ind1],score]
        rowCount +=1


In [27]:
#Checking final result
itemAffinity.head()

item1   item2  score
0  5001.0  5002.0    0.4
1  5002.0  5001.0    0.4
2  5001.0  5003.0    0.0
3  5003.0  5001.0    0.0
4  5001.0  5004.0    0.2

So this is the list that come up with the list of affinity scores. So how do we do recommendations? Let us say the customer bought a product 5,001. We want to find what items want to recommend to him. And for that, We are going to go back to this table, go to all the records that are item one in the first column, and get the list of all the items two and their scores. And we can do that in descending order. And those items that you see here is what I want to recommend.

In [28]:
searchItem=5001
recoList=itemAffinity[itemAffinity.item1==searchItem]\
        [["item2","score"]]\
        .sort_values("score", ascending=[0])
        
print("Recommendations for item 5001\n", recoList)

Recommendations for item 5001
      item2  score
0   5002.0    0.4
6   5005.0    0.4
20  5002.0    0.4
26  5005.0    0.4
40  5002.0    0.4
46  5005.0    0.4
70  5002.0    0.4
76  5005.0    0.4
4   5004.0    0.2
24  5004.0    0.2
44  5004.0    0.2
74  5004.0    0.2
2   5003.0    0.0
22  5003.0    0.0
42  5003.0    0.0
72  5003.0    0.0


So for 5,001, we see that 5,002 and 5,005 has a score of 0.4, of 5,004 has 0.2, and 5,003 has zero. So what we can do is have a threshold that we are going to only recommend those items whose score is above a certain point.
for instance if we have that threshold as 0.25, then we would recommend the products 5,002 and 5,005 to the customer
